http://web.eecs.umich.edu/~mihalcea/downloads.html

In [91]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import semcor
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
import random
import string
from pprint import pprint

### Funzioni usate dall'algoritmo di Lesk

In [92]:
def get_context(sentence):
    # bag of words approach
    sentence.lower()
    tokens = word_tokenize(sentence) 
    tokens = list(filter(lambda token: token not in string.punctuation, tokens))
    wnl = nltk.WordNetLemmatizer()
    ps = PorterStemmer()
    return [ps.stem(wnl.lemmatize(token.lower())) for token in tokens if token not in stopwords.words('english')] #lemma o stemma? solo uno dei due da lasciare

def get_gloss_examples(sense):
    gloss = get_context(sense.definition())
    examples = get_context(' '.join((sense).examples()))
    return gloss + examples

def get_overlap(list_1, list_2):
    return [value for value in list_1 if value in list_2]

### Algoritmo di Lesk

In [93]:
def lesk(word, sentence):
    best_sense = wn.synsets(word)[0]
    max_overlap = 0
    context = get_context(sentence)
    sifnature = None
    overlap = None
    for sense in wn.synsets(word):
        signature = get_gloss_examples(sense)
        overlap = get_overlap(context, signature)
        if len(overlap) > max_overlap:
            max_overlap = len(overlap)
            best_sense = sense
    return best_sense

### Funzioni utili per l'esecuzione dell'algoritmo di Lesk su 50 frasi scelte randomicamente da SemCor

In [135]:
def get_nouns(list_of_sentences):
    nouns = []
    ps = PorterStemmer()
    wnl = WordNetLemmatizer()
    for sentence in list_of_sentences:
        for word in sentence:
            print(f'{word[0]}\n\n\n')
            if 'NN' in str(word) and len(word) == 1 and word[0]: 
            #prendo solo parole singole per evitare i noun phrase e che inizino con la minuscola così evito i nomi propri.
                nouns.append(word)
                break
    return nouns

def get_rand_elems(array):
    rands = random.sample(range(0, len(array)), 50)
    return [array[i] for i in rands]

### Esecuzione

In [95]:
sentence = "we went up to the mountains by car"
print(wn.synsets('car')[4].examples())
lesk('car', sentence)
wn.synsets('group')[2].definition()

['they took a cable car to the top of the mountain']


'a set that is closed, associative, has an identity element and every element has an inverse'

In [97]:
tagged_sentences = semcor.tagged_sents(tag="both")
random_sentences = get_rand_elems(tagged_sentences)

select a NN from every of the 50 sentences randomly selected from semcor 

In [136]:
nouns = get_nouns(random_sentences)
pprint(nouns)

This



(NN phenomenon)



Player



(VB Is)



it



(RB not)



(JJ ironical)



that



(NE (NNP Roger Williams))



The



FBI



Yet



the



press



That



(VB was)



(NE (NNP Bartoli))



This



may



(VB be)



(JJ helpful)



in



(VB improving)



the



(JJ competitive)



(NN position)



After



,



we



(VB began)



to



(VB get)



``



visitors



If



you



(VB use)



(NN parking)



I



(VB was)



suddenly



conscious



of



my



bare



arms



They



(JJ all)



(VB prayed)



(RB now)



that



the



(NN North)



If



(NE (NNP Nikita))



(NE (NNP Hans))



Board



(VB indicated)



Monday



``



A



child



We



(VB like)



to



(VB run)



a



nice



quiet



place



I



(VB looked for)



(NE (NNP Jessica))



There



is



,



to



(VB begin)



,



an



(JJ important)



(NN sex)



The



concept



Though



they



would



(VB produce)



(JJ some)



(RB very)



(JJ memorable)



and



(JJ lasting)



(NN song

In [ ]:
sent = tagged_sentences[0]

AttributeError: 'list' object has no attribute 'label'